# Julia Things

### Environment

First things first. Let us set up the environment with the requried packages for this notebook:

In [1]:
for p in ("Knet", "Plots", "Plotly.jl")
    Pkg.installed(p) == nothing && Pkg.add(p)
end

using Knet, Plots
gr()

Knet.gpu(0); # set the desired GPU to use
atype = Array{Float32}; # atype = KnetArray{Float32} for gpu usage, Array{Float32} for cpu. 

println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))
println("GPU: ", readstring(`nvidia-smi --query-gpu=name --format=csv,noheader`))

OS: Linux
Julia: 0.6.0
Knet: 0.8.5+
GPU: NVS 310
TITAN X (Pascal)



### New Stuff

In this notebook we introduce the following Julia/Knet packages and functions:

* Knet's function [accuracy](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.accuracy): Knet's function `accuracy` allows you to automatically calculate the accuracy of your model. For example, as long as the function `predict` has been defined, and the training/testing data has been processed through Julia's function [minibatch](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.minibatch) such that `dtrn=minibatch(xtrn,ytrn)`, `accuracy` allows you to calculate the accuracy as `acc=accuracy(w,dtrn,predict)`, where `w` is your model (i.e the parameters/weights of your model). 

# Multiclass Logistic Regression 

If you've made it through our tutorials on linear and logistic regression, then you're past the hardest part. You already know how to load and manipulate data, build computation graphs on the fly, and take derivatives. You also know how to define a loss function, construct a model, and write your own optimizer. Nearly all neural networks that we'll build in the real world consist of these same fundamental parts. The main differences will be the type and scale of the data and the complexity of the models. And every year or two, a new hipster optimizer comes around, but at their core they're all subtle variations of stochastic gradient descent.

In [the previous chapter](../chapter02_supervised-learning/logistic-regression.ipynb), we introduced logistic regression, a classic algorithm for performing binary classification. We implemented a model 

$$\hat{y} = \sigma(\boldsymbol{w}^T \boldsymbol{x} + b)$$
where $\sigma$ is the sigmoid squashing function. This activation function on the final layer was crucial because it forced our outputs to take values in the range [0,1]. That allowed us to interpret these outputs as probabilties.
We then updated our parameters to give the true labels (which take values either 1 or 0) the highest probability.
In that tutorial, we looked at predicting whether or not an individual's income exceeded $50k based on features available in 1994 census data, and we also worked with the medical appoinment data. 

Binary classification is quite useful. We can use it to predict spam vs. not spam or cancer vs not cancer. 
But not every problem fits the mold of binary classification. Sometimes we encounter a problem where each example could belong to one of $k$ classes. For example, a photograph might depict a cat or a dog or a zebra or ... (you get the point). Given $K$ classes, the most naive way to solve a *multiclass classification* problem 
is to train $K$ different binary classifiers $f_k(\boldsymbol{x})$. We could then predict that an example $\boldsymbol{x}$ belongs  to the class $k$ for which the probability that the label applies is highest $\max_{k} $f_k$(\boldsymbol{x})$.

There's a smarter way to go about this. We could force the output layer to be a discrete probability distribution over the $K$ classes. To be valid probabity distribution, we'll want the output $\hat{y}$ to (i) contain only non-negative values, and (ii) sum to 1. We accomplish this by using the *softmax* function. Given an input vector $z$, softmax does two things. First, it exponentiates (elementwise) $e^{z}$, forcing all values to be strictly positive.
Then it normalizes so that all values sum to $1$. Following the softmax operation computes the following


$$\mbox{softmax}(\boldsymbol{z}) = \frac{e^{\boldsymbol{z}} }{\sum_{k=1}^K e^{z_i}}$$

Because now we have $K$ outputs and not $1$ we'll need weights connecting each of our inputs to each of our outputs. Graphically, the network looks something like this:

![](../img/simple-softmax-net.png)

We can represent these weights one for each input node, output node pair in a matrix $W$. We generate the linear mapping from inputs to outputs via a matrix-vector product $W \boldsymbol{x} + \boldsymbol{b}$. Note that the bias term is now a vector, with one component for each output node. The whole model, including the ativation function can be written:

$$\hat{y} = \mbox{softmax}(W\boldsymbol{x} + \boldsymbol{b})$$

This model is sometimes called *multiclass logistic regression*. 
Other common names for it include *softmax regression* and *multinomial regression*.
For these concepts to sink in, let's actually implement softmax regression,
and pick a slightly more interesting dataset this time. 

## The MNIST dataset

In this example we build a simple classification model for the [MNIST](http://yann.lecun.com/exdb/mnist/) handwritten digit recognition dataset. MNIST has 60000 training and 10000 test examples. Each input x consists of 784 pixels representing a 28x28 image. The corresponding output indicates the identity of the digit 0..9.

![png](https://jamesmccaffrey.files.wordpress.com/2014/06/firsteightimages.jpg) 

To start, we'll use Knet's utilities for grabbing a copy of this dataset:

In [2]:
include(Knet.dir("data","mnist.jl"));
xtrn, ytrn, xtst, ytst = mnist();

INFO: Loading MNIST...


There are two parts of the dataset for training and testing. Each part has N items and each item is a tuple of an image and a label:

In [3]:
size(xtrn), size(ytrn)

((28, 28, 1, 60000), (60000,))

Note that each image has been formatted as a 3-tuple (height, width, channel). For color images, the channel would have 3 dimensions (red, green and blue). In this case we have a gray scale image with a single channel. Note that `ytrn` is a `Array{UInt8,1}` data type. Let us take a look at these labels:

In [4]:
ytrn[1:10], 1ytrn[1:10]

(UInt8[0x05, 0x0a, 0x04, 0x01, 0x09, 0x02, 0x01, 0x03, 0x01, 0x04], [5, 10, 4, 1, 9, 2, 1, 3, 1, 4])

Ok, let us now use Knet's minibatch function to prepare the date for training:

In [5]:
dtrn = minibatch(xtrn, ytrn, 100; xtype=atype);
dtst = minibatch(xtst, ytst, 100; xtype=atype);

Note that the input images have been reshaped to 784x1

In [6]:
dtrn.xtype, dtrn.ytype

(Array{Float32,N} where N, Array{UInt8,1})

## Model 

### Multiclass logistic regression

Recall from our previous topic on [logistic regression](logistic-regression.ipynb) that for the loss function we can use the *negative log probability:*

$$ l(\theta) = - \sum_i^n\log\big(\mathbb{P}(y_i|\boldsymbol{x}_i)\big)$$

where $y_i\in\mathbb{R}$ and $\boldsymbol{x}_i\in\mathbb{R}^d$. For binary logistic regression, where we deal with only two classes, we've seen that we can write 

$$\mathbb{P}(y_i|\hat{y}_i) = \sigma(\hat{y}_i)^{y_i}(1-\sigma(\hat{y}_i))^{1-y_i}$$

where $\hat{y}_i$ is the probability that  $x_i$ has a given label. In that case we found the loss function to be

$$ l = \sum_{i=1}^n y\log\hat{y}_i + (1-y)\log(1-\hat{y}_i) $$

This equation is the negative log likelihood of the Bernoulli distribution. Now, consider the multiclass case where we apply the softmax function instead of the logistic function:

$$\hat{y}_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$$

For binary classification problems, the softmax function outputs two values (between 0 and 1 and sum to 1) to give the prediction of each class. While the sigmoid function outputs one value (between 0 and 1) to give the prediction of one class (so the other class is 1-p). See [here](https://stats.stackexchange.com/questions/198038/cross-entropy-or-log-likelihood-in-output-layer) for an example. 

Before we can start training, we're going to need to define a loss function that makes sense when our prediction is a  probability distribution. 

The relevant loss function here is called [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy). The cross entropy between two probability distributions $p$ and $q$ is defined as

$$H(p,q)=\operatorname {E}_{p}[-\log q]=H(p)+D_{{{\mathrm  {KL}}}}(p\|q),\!$$

where $H(p,q)$ is the entropy of $p$, and $D_{{{\mathrm  {KL}}}}(p\|q)$ is the  is the Kullback–Leibler divergence of $q$ from $p$. For discrete $p$ and $q$ this means 

$$H(p,q)=\operatorname {E}_{p}[-\log q]=-\sum_{i=1}^n p_i \log q_i$$

Note that this is simply the expected value of $-\log q$ given the probability distribution $p$. For example, for binary labels such that $p=\{0,1\}$ and $q=\{\hat{y}, 1-\hat{y}\}$ we can write

$$H(p,q)=-\sum_{i=1}^n y_i \log  \hat{y} + (1-y_i) \log (1-\hat{y})$$

which is exactly the log loss for binary labels. For more information on this topic see [here](https://jamesmccaffrey.wordpress.com/2016/09/25/log-loss-and-cross-entropy-are-almost-the-same/). 

### Implementation

Assument that $y$ is an $K\times m$ matrix, where $m$ is the number of samples and $K$ is the number of labels. A Naive implementaion of this theory is a follow:

In [1]:
softmax(z) = exp.(z) ./ sum(exp.(z), 1)
cross_entropy(yhat, y) = - sum(y .* log.(yhat), 1)

cross_entropy (generic function with 1 method)

In [8]:
m = 1000;
K = 10;
z = rand(K, m);
yhat = softmax(z);

Let's confirm that indeed all of our rows sum to 1:

In [1]:
mapslices(indmax, p, 2)

LoadError: [91mUndefVarError: p not defined[39m

We could then implement the loss functions as:

In [10]:
cross_entropy(yhat, y) = - sum(sum(z .* log.(z), 1))

cross_entropy (generic function with 1 method)

First, `log.` computes the logarithm of each element of `yhat`. Next, we multiply each element of `y` with the corresponding element of `log(yhat)`. `sum( , 1)` adds the elements in the first dimension of y (to understand this, notice that for binary labels we have to sum $y_i \log  \hat{y} + (1-y_i) \log (1-\hat{y})$ before we sum over the samples). Finally, the last sume computes the sum over all the examples in the batch (sometimes this is a mean instead of a sum). 

Note that in the [source code](https://github.com/denizyuret/Knet.jl/blob/786a3cde09c8699801798af2b89de6dc552ed9be/src/loss.jl#L157-L167), we don't use this formulation, because it is numerically unstable. In Julia, `nll` computes the negative log likelihood of your predictions compared to the correct answers. Roughly, `nll` does something like this:

$$\log(\hat{y}_i) = z_i - \log \big(\sum_{j=1}^K e^{z_j}\big)$$

Let us see how `nll` works. First we create a random linear prediction `z`, and then we run it through `softmax`. The labels are also ramdomized. Do notice tha `nll` do not require one-hot labels. Instead, we simply create an array with random labels from 1 to $K$:

In [11]:
y = Array{UInt8,2}(rand(1:K, 1, m));
z = rand(K, m);
yhat = softmax(rand(K, m));

In [12]:
(typeof(y), size(y)), (typeof(yhat), size(yhat)), (y[1, 1:2] * 1)

((Array{UInt8,2}, (1, 1000)), (Array{Float64,2}, (10, 1000)), [10, 10])

Ok, now let us make sure `nll` gives an expected output. 

In [13]:
nll(yhat, y)

2.3016897033631283

Is this number what we expected? Consider that our original predictions were random chosen. Since there are 10 labels, on average we would be correct 1/10th of the time. The:

In [14]:
-log(1/10)

2.3025850929940455

Ok. It's time to build our model and run our tests:

In [15]:
predict(w,x) = w[1]*mat(x) .+ w[2]
loss(w,x,ygold) = nll(predict(w,x), ygold)
lossgradient = grad(loss)

(::gradfun) (generic function with 1 method)

Here, we assume ygold is an array of N integers indicating the correct answers for N instances (we use ygold=10 to represent the 0 answer) and predict() gives us a (10,N) matrix of scores for each answer. [mat](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.mat) is needed to convert the (28,28,1,N) x array to a (784,N) matrix so it can be used in matrix multiplication. Other than the change of loss function, the softmax model is identical to the linear regression model. We use the same predict (except for mat reshaping), train and set lossgradient=grad(loss) as before.

Now let's train a model on the MNIST data:

In [16]:
function train(w, data; lr=.1)
    for (x,y) in data
        dw = lossgradient(w, x, y)
        for i in 1:length(w)
            w[i] -= lr * dw[i]
        end   
    end

end

train (generic function with 1 method)

In [17]:
w = map(atype, Any[ 0.1f0*randn(Float32, 10, 784), zeros(Float32, 10, 1) ])
w = Any[ 0.1f0*randn(Float32,10,784), zeros(Float32,10,1) ];
println((:epoch, 0, :trn, accuracy(w,dtrn,predict), :tst, accuracy(w,dtst,predict)))
for epoch=1:10
    train(w, dtrn; lr=0.5)
    println((:epoch, epoch, :trn, accuracy(w,dtrn,predict), :tst, accuracy(w,dtst,predict)))
end

(:epoch, 0, :trn, 0.10478333333333334, :tst, 0.1111)
(:epoch, 1, :trn, 0.9003333333333333, :tst, 0.9049)
(:epoch, 2, :trn, 0.9081166666666667, :tst, 0.9096)
(:epoch, 3, :trn, 0.91195, :tst, 0.9111)
(:epoch, 4, :trn, 0.9139833333333334, :tst, 0.9121)
(:epoch, 5, :trn, 0.91535, :tst, 0.9137)
(:epoch, 6, :trn, 0.9167166666666666, :tst, 0.9149)
(:epoch, 7, :trn, 0.9176, :tst, 0.9145)
(:epoch, 8, :trn, 0.9185166666666666, :tst, 0.9156)
(:epoch, 9, :trn, 0.9194166666666667, :tst, 0.9154)
(:epoch, 10, :trn, 0.9196833333333333, :tst, 0.9155)


Ok, let us check the accuracy on the testing and training set:

In [19]:
accuracy(w, dtrn, predict), accuracy(w, dtst, predict)

(0.9196833333333333, 0.9155)

Jeepers. We can get more than 90% accuracy on both data sets at this task just by training a linear model for a few seconds! You might reasonably conclude that this problem is too easy to be taken seriously by experts.
But until recently, many papers (Google Scholar says 13,800) were published using results obtained on this data.

## Next:

[Regularization](section5-regularization.ipynb).


For whinges or inquiries, open an issue on [GitHub](https://github.com/moralesq/Knet-the-Julia-dope).